In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
b=True
while(b):
    print("1. Analyze Data")
    print("2. Edit Data")
    print("3. Sort Folder")
    print("4. Exit")
    choice=input("Enter Choice: ")
    if(choice=="1"):
        print(1)
        pass
    elif(choice=="2"):
        print(2)
        pass
    elif(choice=="3"):
        print(3)
        pass
    elif(choice=="4"):
        print("Exiting Program...")
        b=False
    else:
        print("Please Enter Valid Choice !")


In [ ]:
def Load_Data():
    b=True
    while(b):
        print("1. Analyze CSV File")
        print("2. Analyze Parquet File")
        print("3. Analyze Excel File")
        print("4. Return")
        choice=input("Enter Choice: ")
        if(choice=="1"):
            print(1)
            pass
        elif(choice=="2"):
            print(2)
            pass
        elif(choice=="3"):
          print(3)
          pass
        elif(choice=="4"):
          # print("Exiting Program...")
          b=False
        else:
          print("Please Enter Valid Choice !")


In [ ]:
def Analyze_Data():
    b=True
    while(b):
        print("1. General Analysis")
        print("2. Custom Analysis")
        print("3. Return")
        choice=input("Enter Choice: ")
        if(choice=="1"):
            print(1)
            pass
        elif(choice=="2"):
            print(2)
            pass
        elif(choice=="3"):
          # print("Exiting Program...")
          b=False
        else:
          print("Please Enter Valid Choice !")

In [ ]:
def Edit_Data():
    b = True
while b:
    print("1. Add a New Row")
    print("2. Update an Existing Row")
    print("3. Delete a Row")
    print("4. Edit Specific Column Data")
    print("5. Fill Missing Data")
    print("6. Rename Columns")
    print("7. Change Data Type of a Column")
    print("8. Sort Data")
    print("9. Filter Data")
    print("10. Remove Duplicates")
    print("11. Return")
    
    choice = input("Enter Choice: ")
    
    if choice == "1":
        print("Adding a new row...")
        pass
    elif choice == "2":
        print("Updating an existing row...")
        pass
    elif choice == "3":
        print("Deleting a row...")
        pass
    elif choice == "4":
        print("Editing specific column data...")
        pass
    elif choice == "5":
        print("Filling missing data...")
        pass
    elif choice == "6":
        print("Renaming columns...")
        pass
    elif choice == "7":
        print("Changing data type of a column...")
        pass
    elif choice == "8":
        print("Sorting data...")
        pass
    elif choice == "9":
        print("Filtering data...")
        pass
    elif choice == "10":
        print("Removing duplicates...")
        pass
    elif choice == "11":
        print("Returning to main menu...")
        b = False
    else:
        print("Please Enter a Valid Choice!")
